In [0]:
!kill -9 -1

In [1]:
#librerias
!pip install tqdm
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io
from tqdm import tqdm

# Text processing
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split

# Text 
from keras.utils import np_utils
from keras.layers import Embedding, LSTM
from keras.optimizers import RMSprop
from keras.callbacks import History

## Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dropout, Dense, Input, BatchNormalization#, merge, Merge
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, Sequential

%matplotlib inline

    100% |████████████████████████████████| 51kB 1.7MB/s 
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Using TensorFlow backend.


In [2]:
##### Solo para Colab #####
!pip install -U -q PyDrive
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
##### Para CoLab #####
# Verificacion de contenido de carpetas en el Drive con su respectivo ID
#file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
file_list = drive.ListFile({'q': "'1gX_xAhoYwIwmOVRd33lAuf4pCM3cChB6' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: colabcombinado, id: 1Lo3bVt7cHIoOHVPjrbILOW-8ocWGjH0w
title: 0-ColabCombinado, id: 1WhxbiFAE5vCZyu8xLcxpHNKmdTxrS52O
title: 0-CombinacionesText, id: 1Lk9Ek6fOIAV6v9ysYe8U8beBpvFEtbPfqVb6tyd-_I4
title: Final Project.pptx, id: 17-MkUf4ltoGnoNHS8XhuI2PzG0Igd9YF
title: Combinado.ipynb, id: 1rGKRTd8TTYKGkxvf3_o65MC5hwEK2B6q
title: Resultados, id: 1mUYucRY_uccBGTQUjG08Rg_rtRsj1h9ziLEXgjAeZSA
title: ~$Final Project.pptx, id: 1lP71seVswTdQmD2xuPKNB63sPy-0Q5GQ
title: ImageColab.ipynb, id: 1QI4l_c2wejdKDavNVdmMz6DSP0es2Jf2
title: ProcText (1).ipynb, id: 1k6E6N6jG6PCCwXzgoEKCskST6_HwCYAO
title: Resultados.xlsx, id: 1Lykn5EEY4RTyTesTN6f5d0LDm1AqIuGv
title: pred_genres_img_RF.csv, id: 1l734WEkdZIeCUxGqJw_i0czo1bfm12IR
title: ~$Resultados.xlsx, id: 1jvUOMuytd6qatDNqWg0NKSd_idSORy75
title: ProcText.ipynb, id: 1d6n-hMm0W-Y9nXuUmGbfEACnmFmcUA2Z
title: features.pkl, id: 1gT_mDY7-1O9nwvWjFQj2qaVNhfl0TPFw
title: Final Project, id: 1PIyDb9Q8kx5U-ShQRAAn1dXlVLRIPLs-VtXERlbpPW0
title: TextColab.ipynb,

In [0]:
##### Para CoLab #####
trainDrive = drive.CreateFile({'id': '1SKNGe6FFghASG38LjtUaoTrmozUibRCZ'})
trainDrive.GetContentFile('dataTraining.csv')
dataTraining=pd.read_csv("dataTraining.csv", index_col=0)

testDrive = drive.CreateFile({'id': '1JWWmvheWnofZl2zqY-7Gu2rJprceYB80'})
testDrive.GetContentFile('dataTesting.csv')
dataTesting=pd.read_csv("dataTesting.csv", index_col=0)

In [0]:
file_id = '1_Grjc2zdV2gHA7unPg_I7qTEdM6zl1Ok' # -> cnn_data.zip
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(downloaded['title'])

In [3]:
# Descomprimir archivo
# -o -> Overwrite
!unzip -o images_original.zip

unzip:  cannot find or open images_original.zip, images_original.zip.zip or images_original.zip.ZIP.


In [0]:
# load photo features
def load_photo_features(filename, dataset):
    from pickle import load
    # load all features
    all_features = load(open(filename, 'rb'))
    # filter features
    features = {k: all_features[str(k)] for k in dataset}
    return features

In [0]:
# Funcion para seleccionar una muestra de las imagenes
def sampleData(dataTrain, dataTest, varTrain, varTest):
    ix_train=np.random.choice(dataTrain.shape[0], varTrain)
    ix_test=np.random.choice(dataTest.shape[0], varTest)
    return dataTrain.iloc[ix_train], dataTest.iloc[ix_test]

newDataTraining, newDataTesting  = sampleData(dataTraining, dataTesting, varTrain=100, varTest=50)

In [0]:
file_id = '1gT_mDY7-1O9nwvWjFQj2qaVNhfl0TPFw' 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile(downloaded['title'])

In [58]:
train_features = load_photo_features('features.pkl', ix_train)
len(train_features)

389

In [0]:
train_features_list = []
ix_train_features = []
for key in train_features:
    train_features_list.append(train_features[key])
    ix_train_features.append(key)

In [60]:
len(ix_train_features)

389

In [0]:
train_features_array = np.vstack(train_features_list)
train_features_array = train_features_array.astype('float32')
train_features_array /= 255

In [0]:
max_review_length = 1000

In [63]:
# Funcion para seleccionar los plot de las peliculas
def sampleData(dataTraining, newIndexTrain):
    #ix_train=np.random.choice(dataTrain.shape[0], varTrain)
    #ix_test=np.random.choice(dataTest.shape[0], varTest)
    newDataTraining = dataTraining.iloc[newIndexTrain]
    newDataTraining['genres'] = newDataTraining['genres'].map(lambda x: eval(str(x)))
    le = MultiLabelBinarizer()
    y_genres = le.fit_transform(newDataTraining['genres'])
    #y_genres.shape
    #vect = CountVectorizer(analyzer=split_into_lemmas, ngram_range=(1, 2), max_features=1000, stop_words='english')
    vect = CountVectorizer(analyzer="word", ngram_range=(1, 2), max_features=max_review_length, stop_words='english')
    X_dtm = vect.fit_transform(newDataTraining['plot'])
    return y_genres, X_dtm

y_genres, X_dtm = sampleData(dataTraining, ix_train_features)
y_genres.shape, X_dtm.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


((389, 24), (389, 1000))

In [0]:
def define_model(vocab_size, max_length):
    # feature extractor (encoder)
    inputs1 = Input(shape=(7, 7, 512))
    fe1 = GlobalMaxPooling2D()(inputs1)
    fe2 = Dense(128, activation='relu')(fe1)
    fe3 = RepeatVector(max_length)(fe2)
    # embedding
    inputs2 = Input(shape=(max_length,))
    emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
    emb3 = LSTM(256, return_sequences=True)(emb2)
    emb4 = GlobalMaxPooling1D() (emb3)
    emb5 = Dense(256)(emb4)
    emb6 = Dropout(0.2)(emb5)
    emb7 = TimeDistributed(Dense(128, activation='relu'))(emb6)
    # merge inputs
    merged = concatenate([fe3, emb7])
    # language model (decoder)
    #lm2 = Dense(500, activation='relu')(merged)
    lm2 = LSTM(500)(merged)
    #lm2a = Dropout(0.5)(lm2)
    #lm2a = BatchNormalization()(lm2)
    lm3 = Dense(500, activation='relu')(lm2)
    outputs = Dense(24, activation='sigmoid')(lm3)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())
    #plot_model(model, show_shapes=True, to_file='plot.png')
    return model

In [0]:
## Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#import cv2
from keras.layers import Conv2D, RepeatVector, TimeDistributed, Activation, GlobalMaxPooling2D, MaxPooling2D, Flatten, Dropout, Dense, Input, BatchNormalization#, merge, Merge
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, Sequential

In [0]:
# Text 
from keras.utils import np_utils, plot_model
from keras.layers import Embedding, LSTM, Reshape, concatenate
from keras.optimizers import RMSprop
from keras.callbacks import History
#from pydot import pydot

In [110]:
max_review_length = 1000
top_words = 1000
model = define_model(vocab_size=top_words, max_length=max_review_length)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 1000)         0                                            
__________________________________________________________________________________________________
global_max_pooling2d_8 (GlobalM (None, 512)          0           input_17[0][0]                   
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1000, 50)     50000       input_18[0][0]                   
__________________________________________________________________________________________________
dense_27 (

In [111]:
train_features_array.shape, X_dtm.shape, y_genres.shape, ix_train.shape

((389, 7, 7, 512), (389, 1000), (389, 24), (400,))

In [112]:
model.fit([train_features_array, X_dtm], y_genres, epochs=3, batch_size=50)

Epoch 1/3
389/389 [==============================] - 203s 522ms/step - loss: 7.9397 - acc: 0.2828
Epoch 2/3
389/389 [==============================] - 198s 509ms/step - loss: 7.6441 - acc: 0.2776
Epoch 3/3
389/389 [==============================] - 192s 494ms/step - loss: 7.6216 - acc: 0.2159


In [113]:
test_features = load_photo_features('features.pkl', dataTesting.index)
len(test_features), dataTesting.index.shape

(3383, (3383,))

In [114]:
test_features_list = []
ix_test_features = []
for key in test_features:
    test_features_list.append(test_features[key])
    ix_test_features.append(key)
test_features_array = np.vstack(test_features_list)
test_features_array = test_features_array.astype('float32')
test_features_array /= 255
test_features_array.shape

(3383, 7, 7, 512)

In [115]:
vect = CountVectorizer(analyzer="word", ngram_range=(1, 2), max_features=1000, stop_words='english')
X_dtm_test = vect.fit_transform(dataTesting['plot'])
X_dtm_test.shape

(3383, 1000)

In [0]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict([test_features_array, X_dtm_test])
#y_pred_test_genres = model.predict_proba(images_testing)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_img_RF.csv', index_label='ID')

In [0]:
files.download('pred_genres_img_RF.csv')